In [1]:
import yfinance as yf
import pandas as pd
import datetime
import warnings 
import math
from  bs4 import BeautifulSoup
import requests
import re
import time
import statistics
from selenium import webdriver
warnings.simplefilter('ignore')

In [2]:
columns_name = ['ticker', 'fairprice', 'currentprice', 'date']
watchlist_fcf = pd.DataFrame(columns = columns_name)
watchlist_fcf['profitmargin'] = ((watchlist_fcf.fairprice/watchlist_fcf.currentprice)-1)

In [3]:
def forward_pe(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    current_closing_price = todays_data['Close'][0]
    forward_eps = ticker.info.get('forwardEps')
    forward_pe = current_closing_price/forward_eps
    return forward_pe

def trailing_pe(symbol):
    ticker = yf.Ticker(symbol)
    trailing_pe = ticker.info.get('trailingPE')
    return trailing_pe

#CAPM(benchmark of personal required rate of return)
def capm_rate(symbol,expected_market_return):
    rf_rate = yf.Ticker('^TNX').info.get('regularMarketPrice')*0.01 #we use 10 yr treasury bond yield as our benchmark of risk-free rate
    beta = yf.Ticker(symbol).info.get('beta')
    capm_rate = rf_rate+beta*(expected_market_return-rf_rate)
    return capm_rate


In [4]:
def fcf_valuation(symbol, wacc, perpetualgrowth):
    ticker = yf.Ticker(symbol)
    info = ticker.info
    financials = ticker.financials
    cashflow = ticker.cashflow
    bs = ticker.balance_sheet
    ocf = cashflow.loc['Total Cash From Operating Activities']
    cap_exp = cashflow.loc['Capital Expenditures']

    netprofitmargin = financials.loc['Net Income']/financials.loc['Total Revenue']

    fcf2profitmargin = (ocf+cap_exp)/financials.loc['Net Income']

    avgproftmargin = netprofitmargin.mean()

    avgfcf2profitmargin = fcf2profitmargin.mean()

    revenuegrowth = info.get('revenueGrowth')

    expectedrev = pd.Series()
    expectedrev = [info.get('totalRevenue')*((1+revenuegrowth)**n) for n in range(0,4)]
    #expectedrev

    expectedprofit = [element*avgproftmargin for element in expectedrev]
    
    expectedfcf = pd.Series()
    expectedfcf = [profit*avgfcf2profitmargin for profit in expectedprofit]


    terminalvalue = expectedfcf[-1]*(1+perpetualgrowth)/(wacc-perpetualgrowth)

    todayvalue = pd.Series()
    for n in range(0,4):
        todayvalue.loc[n] = expectedprofit[n]/((1+wacc)**(n+1))
    todayvalue[4] = terminalvalue/((1+wacc)**4)

    cash = bs.loc['Cash'][0]
    debt = bs.loc['Long Term Debt'][0]
    equity_value = todayvalue.sum()+cash-debt
    
    fairvalue = equity_value/info.get('sharesOutstanding')
    return fairvalue, avgproftmargin, avgfcf2profitmargin

def fcf_valuation_adjusted(symbol, wacc, perpetualgrowth, avgproftmargin, avgfcf2profitmargin):
    ticker = yf.Ticker(symbol)
    info = ticker.info
    financials = ticker.financials
    cashflow = ticker.cashflow
    bs = ticker.balance_sheet
    ocf = cashflow.loc['Total Cash From Operating Activities']
    cap_exp = cashflow.loc['Capital Expenditures']

    revenuegrowth = info.get('revenueGrowth')

    expectedrev = pd.Series()
    expectedrev = [info.get('totalRevenue')*((1+revenuegrowth)**n) for n in range(0,4)]
    #expectedrev

    expectedprofit = [element*avgproftmargin for element in expectedrev]
    
    expectedfcf = pd.Series()
    expectedfcf = [profit*avgfcf2profitmargin for profit in expectedprofit]

    terminalvalue = expectedfcf[-1]*(1+perpetualgrowth)/(wacc-perpetualgrowth)

    todayvalue = pd.Series()
    for n in range(0,4):
        todayvalue.loc[n] = expectedprofit[n]/((1+wacc)**(n+1))
    todayvalue[4] = terminalvalue/((1+wacc)**4)

    cash = bs.loc['Cash'][0]
    debt = bs.loc['Long Term Debt'][0]
    equity_value = todayvalue.sum()+cash-debt

    fairvalue = equity_value/info.get('sharesOutstanding')
    return fairvalue
#fcf_valuation('AMZN', 0.056, 0.02)[1]

In [6]:
x = str(input('Please enter ticker: '))
y = float(input('Please enter ticker\'s WACC: '))

ticker = x
wacc = y
perpetualgrowth =0.03 #set 3% as our company's perpetual growth rate
date_object = datetime.date.today()
try:
    result = fcf_valuation(ticker,wacc,perpetualgrowth)
    dict = {'ticker':ticker, 'fairprice':result[0], 'currentprice':yf.Ticker(ticker).info.get('regularMarketPrice'), 'profitmargin': ((result[0]/yf.Ticker(ticker).info.get('regularMarketPrice'))-1), 'date': date_object}
    if any(watchlist_fcf['ticker'] == ticker):
        watchlist_fcf = watchlist_fcf.drop(watchlist_fcf[watchlist_fcf.ticker == ticker], axis=1)
        watchlist_fcf = watchlist_fcf.append(dict, ignore_index=True).dropna()
        print(ticker+' data is already updated.')
    else:
        watchlist_fcf = watchlist_fcf.append(dict, ignore_index=True)
except:
    pass

watchlist_fcf
#watchlist_fcf.to_csv('/Users/we/Desktop/dcf_watchlist.csv')

,ticker,fairprice,currentprice,date,profitmargin
0,AAPL,130.675645,145.38,2022-06-06,-0.101144
